# Predidicting Gentrification using Feed Forward Neural Network

### Importing the Median House Income Data 

In [93]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



### - Rent
DATA_Rent = pd.read_csv('sub-borougharea-medianhouseholdincome2017.csv',na_values=[''])

dt_num_rent = DATA_Rent.loc[:,['2000', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
               '2016']]



### Massaging the data and Feature engineering

In [94]:
dt_chg_rent = dt_num_rent.divide(dt_num_rent.loc[:, '2000'], axis=0)

dt_chg_rent_melt = pd.melt(dt_chg_rent, var_name='Year', value_name='Rent_Pct').drop('Year', 1)
dt_rent = pd.melt(DATA_Rent, id_vars=['short_name', 'long_name', 'Sub-Borough Area'],var_name='Year', value_name='Rent')
dt_rent = pd.concat([dt_rent, dt_chg_rent_melt], axis=1)
yr_chg_mean = dt_chg_rent.mean(0).reset_index()
yr_chg_mean.columns = ['Year', 'yr_mean']
dt_rent = pd.merge(dt_rent, yr_chg_mean, on='Year')
dt_rent['delta'] = dt_rent.loc[:, 'Rent_Pct'] - dt_rent.loc[:, 'yr_mean']
dt_rent['growth_flag'] = (dt_rent['delta'] > 0).astype(int)
sum_consec = 0
for br in dt_rent.loc[:, 'Sub-Borough Area'].unique():
    for yr in ['2000', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015','2016']:
        if float(dt_rent.loc[(dt_rent['Year'] == yr) & (dt_rent['Sub-Borough Area'] == br), 'delta']) > 0:
            sum_consec += 1
        else:
            sum_consec = 0
        dt_rent.loc[(dt_rent['Year'] == yr) & (dt_rent['Sub-Borough Area'] == br), 'cumm_growth_yr'] = sum_consec


### Gentrification Function

In [95]:
def paint(ls):
    rtn = []
    yr = 0
    yr2 = 0
    flag = True
    while len(ls) != 0:
        if ls[-1] >= 4:
            rtn = rtn + list(zip(['Gentrifying'] * int(ls[-1]), range(ls[-1], 0, -1)))
        
            flag = False
            ls = ls[:len(ls) - ls[-1]]
        elif ls[-1] < 4 and flag == True:
            rtn.append(['Gentrified', yr])
            yr += 1
            ls.pop()
        elif ls[-1] < 4 and flag == False:
            rtn.append(['Non-Gentrified', yr2])
            yr2 += 1
            ls.pop()
    rtn.reverse()
    rtn = pd.DataFrame(np.array(rtn), columns=['gentrify_flag', 'passed_years'])
    rtn.passed_years = rtn.passed_years.astype(int)
    rtn.loc[rtn['gentrify_flag'] == 'Non-Gentrified', 'passed_years'] = \
        yr2 - rtn.loc[rtn['gentrify_flag'] == 'Non-Gentrified', 'passed_years']
    rtn.loc[rtn['gentrify_flag'] == 'Gentrified', 'passed_years'] = \
        yr - rtn.loc[rtn['gentrify_flag'] == 'Gentrified', 'passed_years']
    return rtn

range(ls[-1], 0, -1)


range(12, 0, -1)

In [96]:
dt_rent = dt_rent.sort_values(by=['Sub-Borough Area', 'Year']).reset_index(drop=True)
dt_rent.head()

,short_name,long_name,Sub-Borough Area,Year,Rent,Rent_Pct,yr_mean,delta,growth_flag,cumm_growth_yr
0,hh_inc_med_adj,Median household income ($2017),Astoria,2000,53924.701370,1.000000,1.000000,0.000000,0,0.0
1,hh_inc_med_adj,Median household income ($2017),Astoria,2005,46660.000000,0.865281,0.941800,-0.076519,0,0.0
2,hh_inc_med_adj,Median household income ($2017),Astoria,2006,51237.801359,0.950173,0.987463,-0.037290,0,0.0
3,hh_inc_med_adj,Median household income ($2017),Astoria,2007,53910.000000,0.999727,0.993594,0.006133,1,1.0
4,hh_inc_med_adj,Median household income ($2017),Astoria,2008,59294.152225,1.099573,1.024123,0.075450,1,2.0


In [97]:
dt_rent = dt_rent.sort_values(by=['Sub-Borough Area', 'Year']).reset_index(drop=True)
tmp = pd.DataFrame([])
for br in dt_rent.loc[:, 'Sub-Borough Area'].unique():
    ls = dt_rent.loc[dt_rent['Sub-Borough Area'] == br, 'cumm_growth_yr'].astype(int).tolist()
    tmp = pd.concat([tmp, paint(ls)])

dt_rent = pd.concat([dt_rent, tmp.reset_index(drop=True)], axis=1)
dt_rent.head()

,short_name,long_name,Sub-Borough Area,Year,Rent,Rent_Pct,yr_mean,delta,growth_flag,cumm_growth_yr,gentrify_flag,passed_years
0,hh_inc_med_adj,Median household income ($2017),Astoria,2000,53924.701370,1.000000,1.000000,0.000000,0,0.0,Non-Gentrified,1
1,hh_inc_med_adj,Median household income ($2017),Astoria,2005,46660.000000,0.865281,0.941800,-0.076519,0,0.0,Non-Gentrified,2
2,hh_inc_med_adj,Median household income ($2017),Astoria,2006,51237.801359,0.950173,0.987463,-0.037290,0,0.0,Non-Gentrified,3
3,hh_inc_med_adj,Median household income ($2017),Astoria,2007,53910.000000,0.999727,0.993594,0.006133,1,1.0,Non-Gentrified,4
4,hh_inc_med_adj,Median household income ($2017),Astoria,2008,59294.152225,1.099573,1.024123,0.075450,1,2.0,Non-Gentrified,5


### Importing and cleaning Median Household Income data

In [105]:
### - Income
DATA_Income = pd.read_csv('sub-borougharea-medianhouseholdincome2017.csv',
                          na_values=[''],
                          )

dt_num_income = DATA_Income.loc[:,
                ['2000', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
                 '2016']]


dt_pvt_flag = pd.concat([DATA_Income.loc[:, 'Sub-Borough Area'],
                         dt_num_income.loc[:, '2000'] <= dt_num_income.loc[:, '2000'].quantile(0.4)
                         ], axis=1)
dt_pvt_flag.columns = ['Sub-Borough Area', 'Poverty_flag']

dt_income = pd.melt(DATA_Income, id_vars=['short_name', 'long_name', 'Sub-Borough Area'],
                    var_name='Year', value_name='Income')
dt_income = pd.merge(dt_income, dt_pvt_flag, on='Sub-Borough Area')
# dt_income = pd.concat([dt_income, dt_chg_rent_melt], axis=1)
dt_income = dt_income.sort_values(by=['Sub-Borough Area', 'Year']).reset_index(drop=True)

dt_income_rent = pd.concat([dt_rent, dt_income], axis=1)
dt_income_rent = dt_income_rent.iloc[:, list(range(2, 12)) + [16, 17]]
# dt_income_rent = pd.merge(dt_income_rent, dt_pvt_flag, on='Sub-Borough Area')

for br in dt_income_rent.loc[dt_income_rent['Poverty_flag'] == True, 'Sub-Borough Area'].unique():
    if len(dt_income_rent.loc[(dt_income_rent['Sub-Borough Area'] == br) &
                              (dt_income_rent['gentrify_flag'] == 'Gentrified'), :]) == 13:
        dt_income_rent.loc[dt_income_rent['Sub-Borough Area'] == br, 'gentrify_flag'] = 'Non-Gentrified'



DATA_Income.head()

,short_name,long_name,Sub-Borough Area,2000,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,hh_inc_med_adj,Median household income ($2017),Greenwich Village/Financial District,101669.764384,115100,115427.099773,121820,120389.962932,115417.396305,113844.301681,129931.689744,109362.782397,131013.186467,124018.824886,125884.659231,139895.693050
1,hh_inc_med_adj,Median household income ($2017),Lower East Side/Chinatown,42668.931507,39690,44894.061985,37180,41611.250053,46775.146543,47172.492693,42450.610129,43401.101873,41959.223153,42683.773277,43559.604234,42013.569153
2,hh_inc_med_adj,Median household income ($2017),Chelsea/Clinton/Midtown,80923.835616,87400,88461.949071,92130,90348.275780,92710.122833,86783.161452,90002.404347,91717.862765,105214.322926,101700.802060,105401.298444,103537.698631
3,hh_inc_med_adj,Median household income ($2017),Stuyvesant Town/Turtle Bay,100051.287671,106650,109152.710829,112480,122805.460807,111910.450839,101582.396705,110075.444497,98260.026605,112994.965756,108989.835914,110117.129545,112561.944526
4,hh_inc_med_adj,Median household income ($2017),Upper West Side,97108.602740,104140,110270.834436,100740,117308.238627,119920.980978,100334.913146,99899.091709,97609.424676,103790.346100,109563.548092,104126.498822,113447.953980


### Merging all the the CSV files

In [ ]:
DATA_Features = pd.read_csv(data_dir + '/datF.csv', )
dt_feat = DATA_Features.drop(['y'], axis=1)
dt_feat.loc[:, 'variable'] = dt_feat.loc[:, 'variable'].str.replace('X', '')

dt_mrg = pd.merge(dt_income_rent, dt_feat, left_on=['Sub-Borough Area', 'Year'],
                  right_on=['Sub.Borough.Area', 'variable'])
dt_mrg = dt_mrg.drop(dt_mrg.columns[12:15], 1)

### Implementation of Feed Forward Neural network 

In [85]:

dt_mrg['gentrify_flag'].replace(['Gentrified', 'Gentrifing', 'Non-Gentrified'], [2,1,0],inplace=True)
dt_mrg['Poverty_flag'].replace([False, True], [0,1],inplace=True)

location = [i for i in range(len(sorted(list(set(dt_mrg['Sub-Borough Area'])))))]
dt_mrg['Sub-Borough Area'].replace(sorted(list(set(dt_mrg['Sub-Borough Area']))),location,inplace=True)

In [86]:
#y_label = list(dt_mrg['gentrify_flag'])
onehot = pd.get_dummies(dt_mrg['gentrify_flag'])
#target_labels = onehot.columns
y_label = onehot.as_matrix()
# y_label = dt_mrg['gentrify_flag']

dt_mrg.drop(['gentrify_flag'],axis=1,inplace=True)
dt_mrg.drop(dt_mrg.columns[[0]],axis=1,inplace=True)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [87]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from sklearn.model_selection import train_test_split

y_encoded = to_categorical(y_label)

x_train, x_test, y_train, y_test = train_test_split(dt_mrg, y_label, test_size=0.2, random_state=0)
model = Sequential()
model.add(Dense(48, input_dim=24, init='uniform', activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(24, init='uniform', activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(3, init='uniform', activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, nb_epoch=500, batch_size=50)
train_acc = model.evaluate(x_train, y_train)
print("%s: %.2f%%" % (model.metrics_names[1], train_acc[1]*100))
    
test_acc = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], test_acc[1]*100))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(48, input_dim=24, activation="relu", kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(24, activation="relu", kernel_initializer="uniform")`
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="softmax", kernel_initializer="uniform")`
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/500
291/291 [==============================] - 0s 1ms/step - loss: 7.1670 - acc: 0.5097
Epoch 2/500
291/291 [==============================] - 0s 71us/step - loss: 2.9369 - acc: 0.6289
Epoch 3/500
291/291 [==============================] - 0s 42us/step - loss: 1.9870 - acc: 0.6220
Epoch 4/500
291/291 [==============================] - 0s 85us/step - loss: 1.8946 - acc: 0.6105
Epoch 5/500
291/291 [==============================] - 0s 68us/step - loss: 1.8484 - acc: 0.6289
Epoch 6/500
291/291 [==============================] - 0s 85us/step - loss: 1.7716 - acc: 0.6678
Epoch 7/500
291/291 [==============================] - 0s 66us/step - loss: 1.6036 - acc: 0.6518
Epoch 8/500
291/291 [==============================] - 0s 100us/step - loss: 1.5615 - acc: 0.6472
Epoch 9/500
291/291 [==============================] - 0s 87us/step - loss: 1.5139 - acc: 0.6838
Epoch 10/500
291/291 [==============================] - 0s 58us/step - loss: 1.4779 - acc: 0.7411
Epoch 11/500
291/291 [=======

291/291 [==============================] - 0s 42us/step - loss: 1.4782 - acc: 0.7755
Epoch 85/500
291/291 [==============================] - 0s 56us/step - loss: 1.4611 - acc: 0.7549
Epoch 86/500
291/291 [==============================] - 0s 60us/step - loss: 1.4174 - acc: 0.7915
Epoch 87/500
291/291 [==============================] - 0s 45us/step - loss: 1.4207 - acc: 0.7915
Epoch 88/500
291/291 [==============================] - 0s 62us/step - loss: 1.4147 - acc: 0.8030
Epoch 89/500
291/291 [==============================] - 0s 57us/step - loss: 1.4889 - acc: 0.7640
Epoch 90/500
291/291 [==============================] - 0s 67us/step - loss: 1.4348 - acc: 0.7869
Epoch 91/500
291/291 [==============================] - 0s 51us/step - loss: 1.4234 - acc: 0.7869
Epoch 92/500
291/291 [==============================] - 0s 63us/step - loss: 1.4184 - acc: 0.8007
Epoch 93/500
291/291 [==============================] - 0s 64us/step - loss: 1.4263 - acc: 0.7915
Epoch 94/500
291/291 [===========

291/291 [==============================] - 0s 63us/step - loss: 1.4234 - acc: 0.7732
Epoch 168/500
291/291 [==============================] - 0s 48us/step - loss: 1.4212 - acc: 0.7847
Epoch 169/500
291/291 [==============================] - 0s 52us/step - loss: 1.4482 - acc: 0.7732
Epoch 170/500
291/291 [==============================] - 0s 59us/step - loss: 1.4486 - acc: 0.7755
Epoch 171/500
291/291 [==============================] - 0s 40us/step - loss: 1.4354 - acc: 0.7824
Epoch 172/500
291/291 [==============================] - 0s 54us/step - loss: 1.4410 - acc: 0.7755
Epoch 173/500
291/291 [==============================] - 0s 53us/step - loss: 1.4159 - acc: 0.7984
Epoch 174/500
291/291 [==============================] - 0s 46us/step - loss: 1.4172 - acc: 0.8007
Epoch 175/500
291/291 [==============================] - 0s 43us/step - loss: 1.4203 - acc: 0.7938
Epoch 176/500
291/291 [==============================] - 0s 60us/step - loss: 1.4261 - acc: 0.7938
Epoch 177/500
291/291 [=

291/291 [==============================] - 0s 48us/step - loss: 1.4307 - acc: 0.7892
Epoch 250/500
291/291 [==============================] - 0s 46us/step - loss: 1.4231 - acc: 0.8053
Epoch 251/500
291/291 [==============================] - 0s 46us/step - loss: 1.4204 - acc: 0.7778
Epoch 252/500
291/291 [==============================] - 0s 48us/step - loss: 1.4118 - acc: 0.7984
Epoch 253/500
291/291 [==============================] - 0s 56us/step - loss: 1.4047 - acc: 0.8053
Epoch 254/500
291/291 [==============================] - 0s 56us/step - loss: 1.4053 - acc: 0.8053
Epoch 255/500
291/291 [==============================] - 0s 44us/step - loss: 1.4036 - acc: 0.8076
Epoch 256/500
291/291 [==============================] - 0s 40us/step - loss: 1.4129 - acc: 0.7824
Epoch 257/500
291/291 [==============================] - 0s 47us/step - loss: 1.4434 - acc: 0.7686
Epoch 258/500
291/291 [==============================] - 0s 50us/step - loss: 1.4434 - acc: 0.7755
Epoch 259/500
291/291 [=

291/291 [==============================] - 0s 46us/step - loss: 0.4262 - acc: 0.8007
Epoch 332/500
291/291 [==============================] - 0s 42us/step - loss: 0.4233 - acc: 0.8087
Epoch 333/500
291/291 [==============================] - 0s 47us/step - loss: 0.4353 - acc: 0.8099
Epoch 334/500
291/291 [==============================] - 0s 36us/step - loss: 0.4532 - acc: 0.7915
Epoch 335/500
291/291 [==============================] - 0s 46us/step - loss: 0.4524 - acc: 0.8064
Epoch 336/500
291/291 [==============================] - 0s 40us/step - loss: 0.4840 - acc: 0.8087
Epoch 337/500
291/291 [==============================] - 0s 45us/step - loss: 0.4748 - acc: 0.7824
Epoch 338/500
291/291 [==============================] - 0s 48us/step - loss: 0.4411 - acc: 0.8076
Epoch 339/500
291/291 [==============================] - 0s 40us/step - loss: 0.4320 - acc: 0.7995
Epoch 340/500
291/291 [==============================] - 0s 37us/step - loss: 0.4307 - acc: 0.8144
Epoch 341/500
291/291 [=

291/291 [==============================] - 0s 46us/step - loss: 0.3981 - acc: 0.8202
Epoch 414/500
291/291 [==============================] - 0s 39us/step - loss: 0.3988 - acc: 0.8236
Epoch 415/500
291/291 [==============================] - 0s 44us/step - loss: 0.3947 - acc: 0.8144
Epoch 416/500
291/291 [==============================] - 0s 39us/step - loss: 0.3968 - acc: 0.8328
Epoch 417/500
291/291 [==============================] - 0s 41us/step - loss: 0.3998 - acc: 0.8121
Epoch 418/500
291/291 [==============================] - 0s 64us/step - loss: 0.4013 - acc: 0.8202
Epoch 419/500
291/291 [==============================] - 0s 57us/step - loss: 0.3898 - acc: 0.8282
Epoch 420/500
291/291 [==============================] - 0s 55us/step - loss: 0.3896 - acc: 0.8236
Epoch 421/500
291/291 [==============================] - 0s 47us/step - loss: 0.3926 - acc: 0.8247
Epoch 422/500
291/291 [==============================] - 0s 48us/step - loss: 0.4012 - acc: 0.8099
Epoch 423/500
291/291 [=

291/291 [==============================] - 0s 38us/step - loss: 0.3907 - acc: 0.8316
Epoch 496/500
291/291 [==============================] - 0s 48us/step - loss: 0.3786 - acc: 0.8328
Epoch 497/500
291/291 [==============================] - 0s 40us/step - loss: 0.4340 - acc: 0.8121
Epoch 498/500
291/291 [==============================] - 0s 44us/step - loss: 0.4444 - acc: 0.8156
Epoch 499/500
291/291 [==============================] - 0s 50us/step - loss: 0.3832 - acc: 0.8293
Epoch 500/500
291/291 [==============================] - 0s 198us/step
acc: 85.57%
73/73 [==============================] - 0s 75us/step
acc: 79.45%


## Accuracy 85.57% and 79.45%